In [1]:
#!pip install datasets

In [2]:
## Please install torch and datasets
import torch
from torchvision.transforms import functional as t
import torch.nn.functional as f
from datasets import load_dataset
import matplotlib.pyplot as plt

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [3]:
## Loading our dataset
ds = load_dataset("ylecun/mnist")

In [4]:
## Data splits

X_train_p = ds["train"]["image"]
Y_train = ds["train"]["label"]
X_test_p = ds["test"]["image"]
Y_test = ds["test"]["label"]

In [5]:
## PIL to Tensors

X_train = [t.pil_to_tensor(x) for x in X_train_p]
X_test = [t.pil_to_tensor(x) for x in X_test_p]
X_train = torch.stack(X_train).to(device)
X_test = torch.stack(X_test).to(device)
print(X_train.shape, X_test.shape)

torch.Size([60000, 1, 28, 28]) torch.Size([10000, 1, 28, 28])


In [6]:
## Fixing the shape

X_train = X_train.view(-1, 28, 28)
X_test = X_test.view(-1, 28, 28)
print(X_train.shape, X_test.shape)

torch.Size([60000, 28, 28]) torch.Size([10000, 28, 28])


In [7]:
## Making labels into tensors

Y_train = torch.tensor(Y_train).to(device)
Y_test = torch.tensor(Y_test).to(device)
Y_train_onehot = f.one_hot(Y_train, num_classes=10).float()
print(Y_train.shape, Y_test.shape, Y_train_onehot.shape)

torch.Size([60000]) torch.Size([10000]) torch.Size([60000, 10])


In [8]:
## Flattening the image as DNN takes flat tensor as input

X_train = X_train.view(-1, 784).float() / 255.0
X_test = X_test.view(-1, 784).float() / 255.0
print(X_train.shape, X_test.shape)

torch.Size([60000, 784]) torch.Size([10000, 784])


In [9]:

import numpy as np
from numpy.linalg import svd



class Linear():
  def __init__(self, input_dims, output_dims, B=True, last=False):
    self.training = True
    self.W = (torch.randn(input_dims, output_dims) * (5/3) / (input_dims**0.5)).to(device) if not last else (torch.randn(input_dims, output_dims) * (5/3) / (input_dims**0.5) * 0.1).to(device)
    if B: self.B = torch.randn(output_dims).to(device) if not last else (torch.randn(output_dims) * 0.1).to(device)
    else: self.B = torch.tensor([]).to(device)

  def __call__(self, x):
    if not torch.equal(self.B, torch.tensor([]).to(device)): self.result = x@self.W + self.B
    else: self.result = x@self.W
    return self.result

  def parameters(self):
    return [self.W] + [self.B]


class Tanh():
  def __init__(self):
    self.training = True
    return None

  def __call__(self, x):
    self.result = torch.tanh(x)
    return self.result

  def parameters(self):
    return []

class Dropout():
  def __init__(self, batch_size, output_dims, rate=0.9):
    self.training = True
    self.rate = rate
    self.factor = (torch.rand(batch_size, output_dims) < self.rate).int().to(device)
    return None

  def __call__(self, x):
    if self.training: self.result = x * self.factor
    else: self.result = x
    return self.result

  def parameters(self):
    return []


def decomposition(A, k=1):

    # SVD
    U, S, VT = svd(A, full_matrices=False)


    # Truncate to rank-k
    U_k = U[:, :k]                # (784 x k)
    S_k = np.diag(S[:k])          # (k x k)
    VT_k = VT[:k, :]              # (k x 10)

    # Factor A_k = L @ R, where L and R are low-rank factors
    sqrt_S_k = np.sqrt(S_k)       # (k x k)
    L = U_k @ sqrt_S_k            # (784 x k)
    R = sqrt_S_k @ VT_k           # (k x 10)

    L_flat = L.flatten()
    R_flat = R.flatten()
    LR_concat = np.concatenate([L_flat, R_flat])

    return LR_concat


In [10]:
batch_size = 256

mid_dims = 3920

W_u1 = (torch.randn(7860, mid_dims) * (5/3) / (7840**0.5)).to(device)
W_u2 = (torch.randn(mid_dims, 7840) * (5/3) / (mid_dims**0.5)).to(device)
W_p_t_1 = (torch.randn(784, 10) * (5/3) / (784**0.5)).to(device)

W_u1.requires_grad = True
W_u2.requires_grad = True
W_p_t_1.requires_grad = False

updaterparams = [W_u1, W_u2]

print(f"Updater parameters: {W_u1.numel() + W_u2.numel()}, Predictor parameters: {W_p_t_1.numel()}")


Y_p_t_1 = torch.zeros(batch_size, 10).to(device)
losses = []

Updater parameters: 61544000, Predictor parameters: 7840


In [11]:
## Training loop for updater and predictor
iters = 4000
alpha = 0.008


for c in range(iters):
    # Pick a random batch for this iteration
    batch_indices = torch.randperm(X_train.size(0))[:batch_size]
    X_batch = X_train[batch_indices]
    Y_batch_onehot = Y_train_onehot[batch_indices]
    Y_batch = Y_train[batch_indices]
    #print(X_batch.shape, Y_batch_onehot.shape, Y_batch.shape, flat_W_p_t_detached.unsqueeze(0).expand(batch_size, 7840).shape, Y_batch.detach().shape, Y_batch_onehot.detach().shape)

    # Zero updater gradients
    for p in updaterparams:
        if p.grad is not None:
            p.grad.zero_()

    # Zero predictor gradients
    if W_p_t_1.grad is not None:
        W_p_t_1.grad.zero_()

    W_p_t_detached = W_p_t_1.detach()
    flat_W_p_t_detached = W_p_t_detached.view(-1)
    x = torch.cat((flat_W_p_t_detached.unsqueeze(0).expand(batch_size, 7840), Y_p_t_1.detach(), Y_batch_onehot.detach()), dim=1)
    x = torch.tanh(x @ W_u1)
    flat_W_p_t_1 = torch.tanh(x @ W_u2)
    flat_W_p_t_1 = flat_W_p_t_1.mean(dim=0)  # Average over the batch
    W_p_t_1 = flat_W_p_t_1.view(784, 10)

    Y_p_t_1 = torch.tanh(X_batch @ W_p_t_1)
    loss = f.cross_entropy(Y_p_t_1, Y_batch)

    loss.backward()

    for p in updaterparams:
        p.data -= alpha * p.grad

    if c % (iters // 100) == 0:
        print(f"Iteration {c:4d}, Loss: {loss.item():.6f}")
        losses.append(loss.item())



Iteration    0, Loss: 2.536762


/tmp/ipykernel_548508/2130484074.py:20: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  if W_p_t_1.grad is not None:


Iteration   40, Loss: 2.830564
Iteration   80, Loss: 2.715928
Iteration  120, Loss: 2.690230
Iteration  160, Loss: 2.821527
Iteration  200, Loss: 2.384098
Iteration  240, Loss: 2.500139
Iteration  280, Loss: 2.586493
Iteration  320, Loss: 2.656035
Iteration  360, Loss: 2.622307
Iteration  400, Loss: 2.546935
Iteration  440, Loss: 2.613420
Iteration  480, Loss: 2.800249
Iteration  520, Loss: 2.588357
Iteration  560, Loss: 2.505977
Iteration  600, Loss: 2.238578
Iteration  640, Loss: 1.654592
Iteration  680, Loss: 1.284471
Iteration  720, Loss: 1.219195
Iteration  760, Loss: 1.127989
Iteration  800, Loss: 1.103439
Iteration  840, Loss: 1.073142
Iteration  880, Loss: 1.150408
Iteration  920, Loss: 1.091032
Iteration  960, Loss: 1.138247
Iteration 1000, Loss: 1.071779
Iteration 1040, Loss: 1.077584
Iteration 1080, Loss: 1.097980
Iteration 1120, Loss: 1.062259
Iteration 1160, Loss: 1.059123
Iteration 1200, Loss: 1.072796
Iteration 1240, Loss: 1.044479
Iteration 1280, Loss: 1.075699
Iteratio

In [12]:
# ## Training loop for updater and predictor with Adam optimizer
# iters = 4000
# alpha = 0.008

# # Create Adam optimizer for updater parameters
# adam_optimizer = torch.optim.Adam(updaterparams, lr=alpha)

# for c in range(iters):
#     # Zero updater gradients
#     adam_optimizer.zero_grad()

#     # Zero predictor gradients
#     if W_p_t_1.grad is not None:
#         W_p_t_1.grad.zero_()

#     W_p_t_detached = W_p_t_1.detach()
#     flat_W_p_t_detached = W_p_t_detached.view(-1)
#     x = torch.cat((flat_W_p_t_detached.unsqueeze(0).expand(batch_size, 7840), Y_p_t_1.detach(), Y_train_onehot.detach()), dim=1)
#     x = torch.tanh(x @ W_u1)
#     flat_W_p_t_1 = torch.tanh(x @ W_u2)
#     flat_W_p_t_1 = flat_W_p_t_1.mean(dim=0)  # Average over the batch
#     W_p_t_1 = flat_W_p_t_1.view(784, 10)

#     Y_p_t_1 = torch.tanh(X_train @ W_p_t_1)
#     loss = f.cross_entropy(Y_p_t_1, Y_train)

#     loss.backward()

#     # Use Adam optimizer step instead of manual SGD
#     adam_optimizer.step()

#     if c % (iters // 1000) == 0:
#         print(f"Iteration {c:4d}, Loss: {loss.item():.6f}")
#         losses.append(loss.item())


In [17]:
def accuracy(X, Y):
    x = torch.tanh(X @ W_p_t_1)
    probs = f.softmax(x, 1)
    answers = x.argmax(1)
    c = 0
    for a, y in zip(answers, Y):
        if a==y: c+=1
    return c / answers.shape[0] * 100

def loss(X, Y):
    x = torch.tanh(X @ W_p_t_1)
    return f.cross_entropy(x, Y)

print(f"train accuracy: {accuracy(X_train, Y_train)} | test accuracy: {accuracy(X_test, Y_test)}")
print(f"train loss: {loss(X_train, Y_train)} | test loss: {loss(X_test, Y_test)}")

train accuracy: 89.30166666666666 | test accuracy: 89.64
train loss: 1.0046555995941162 | test loss: 1.005833387374878


In [14]:
# import os

# plt.figure(figsize=(8, 4))
# plt.plot(losses)
# plt.xlabel('Iteration')
# plt.ylabel('Loss')
# plt.title('Training Loss')

# # Get current notebook filename and save as PNG
# notebook_path = 'model_descent_weight_enc_dec_compression_3920_4000it'
# filename = os.path.splitext(notebook_path)[0] + '.png'
# plt.savefig(filename)
# plt.show()

In [15]:
# batch_size = 60000
# #loss1iters = []


# for midf in range(24, 34):
#     mid_dims = (midf+1) * 196
#     W_u1 = (torch.randn(7840, mid_dims) * (5/3) / (7840**0.5)).to(device)
#     W_u2 = (torch.randn(mid_dims, 7840) * (5/3) / (mid_dims**0.5)).to(device)
#     W_p_t_1 = (torch.randn(784, 10) * (5/3) / (784**0.5)).to(device)

#     W_u1.requires_grad = True
#     W_u2.requires_grad = True
#     W_p_t_1.requires_grad = False

#     updaterparams = [W_u1, W_u2]

#     print(f"Updater parameters: {W_u1.numel() + W_u2.numel()}, Predictor parameters: {W_p_t_1.numel()}")

#     losses = []

#     ## Training loop for updater and predictor
#     alpha = 0.008

#     c = -1
#     while True:
#         c += 1

#         # Zero updater gradients
#         for p in updaterparams:
#             if p.grad is not None:
#                 p.grad.zero_()

#         # Zero predictor gradients
#         if W_p_t_1.grad is not None:
#             W_p_t_1.grad.zero_()

#         W_p_t_detached = W_p_t_1.detach()
#         flat_W_p_t_detached = W_p_t_detached.view(-1)
#         x = torch.tanh(flat_W_p_t_detached @ W_u1)
#         flat_W_p_t_1 = torch.tanh(x @ W_u2)
#         W_p_t_1 = flat_W_p_t_1.view(784, 10)

#         Y_p_t_1 = torch.tanh(X_train @ W_p_t_1)
#         loss = f.cross_entropy(Y_p_t_1, Y_train)

#         loss.backward()

#         for p in updaterparams:
#             p.data -= alpha * p.grad

#         if c>6000: 
#             print("Couldn't find a solution in 6000 iterations")
#             break
#         if loss.item() < 1.05:
#             print("Found a solution in", c, "iterations with mid_dims =", mid_dims)
#             loss1iters.append([mid_dims, c])
#             break


In [16]:
# Plotting loss1iters with (midf+1)*392 on x-axis and c on y-axis
import matplotlib.pyplot as plt

if loss1iters:
    loss1iters_arr = np.array(loss1iters)
    plt.figure(figsize=(6,4))
    plt.plot(loss1iters_arr[:,0], loss1iters_arr[:,1], marker='o')
    plt.xlabel('Mid Dims')
    plt.ylabel('Iterations to reach loss < 1.0')
    plt.title('Iterations vs. Hidden Layer Size')
    plt.grid(True)
    plt.show()
else:
    print("loss1iters is empty.")

NameError: name 'loss1iters' is not defined